In [71]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader
from torchsummary import summary
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score


device = 'mps' if torch.mps.is_available() else 'cpu'
print(device)

mps


In [42]:
data_df = pd.read_csv("clean_data.csv")
data_df.dropna(inplace=True)
original_df = data_df.copy()

In [ ]:
x = data_df.drop(["blue_score", "red_score"], axis=1).values
y = data_df[["blue_score", "red_score"]].values
x

In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15,)


In [72]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)

In [73]:
class dataset(Dataset):
    def __init__(self, x, y):
        self.x = torch.tensor(x, dtype=torch.float32).to(device)
        self.y = torch.tensor(y, dtype=torch.float32).to(device)
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

In [74]:
train_data = dataset(x_train, y_train)
test_data = dataset(x_test, y_test)

In [109]:
BATCH_SIZE = 16

In [110]:
train_dl = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dl = DataLoader(test_data, batch_size=BATCH_SIZE, )

In [111]:
class MyNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2) 
        )

    def forward(self, x):
        return self.model(x)


In [112]:
INPUT_DIM = x_train.shape[1]
model = MyNet(INPUT_DIM).to(device)

In [113]:
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-4)

In [114]:
def mean_absolute_error(y_true, y_pred):
    return torch.mean(torch.abs(y_true - y_pred)).item()

def regression_accuracy(y_true, y_pred, threshold=5.0):
    correct = torch.sum(torch.all(torch.abs(y_true - y_pred) < threshold, dim=1)).item()
    total = y_true.shape[0]
    return correct / total


In [ ]:
EPOCHS = 100

total_loss_train_plot = []
total_loss_val_plot = []
total_acc_train_plot = []
total_acc_val_plot = []

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0
    running_mae = 0.0

    for inputs, labels in train_dl:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_mae += mean_absolute_error(labels, outputs) * inputs.size(0)
        
    epoch_loss = running_loss / len(train_dl.dataset)
    epoch_mae = running_mae / len(train_dl.dataset)
    
    print(f'Epoch {epoch+1}/{EPOCHS}, Loss (MSE): {epoch_loss:.4f}, MAE: {epoch_mae:.4f}')



In [107]:
model.eval()
with torch.no_grad():
    test_loss = 0.0
    test_mae = 0.0
    all_true = []
    all_preds = []
    for inputs, labels in test_dl:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item() * inputs.size(0)
        test_mae += mean_absolute_error(labels, outputs) * inputs.size(0)
        all_true.append(labels)
        all_preds.append(outputs)
        
    test_loss /= len(test_dl.dataset)
    test_mae /= len(test_dl.dataset)
    
      
    all_true = torch.cat(all_true, dim=0)
    all_preds = torch.cat(all_preds, dim=0)
    acc = regression_accuracy(all_true, all_preds, threshold=5.0)
    
    print(f'Test Loss (MSE): {test_loss:.4f}, Test MAE: {test_mae:.4f}, Regression Accuracy (error<{5.0}): {acc*100:.2f}%')


Test Loss (MSE): 1120.9338, Test MAE: 24.5247, Regression Accuracy (error<5.0): 2.10%


In [157]:
def inferencia(fila, model, scaler, threshold=5.0):
   

    features = fila.drop(labels=["blue_score", "red_score"]).values.reshape(1, -1)

    features_scaled = scaler.transform(features)

    tensor_features = torch.tensor(features_scaled, dtype=torch.float32).to(device)

    model.eval()
    with torch.no_grad():
        prediction = model(tensor_features)

    predicted_scores = prediction.cpu().numpy().flatten()

    true_scores = fila[["blue_score", "red_score"]].values.astype(np.float32)

    error_abs = np.abs(true_scores - predicted_scores)

    accuracy = 100.0 if (error_abs < threshold).all() else 0.0

    actual_winner = "blue" if true_scores[0] > true_scores[1] else "red"
    
    pred_winner = "blue" if predicted_scores[0] > predicted_scores[1] else "red"
    # print(
    #     f"Predicción -> blue_score: {predicted_scores[0]:.2f}, red_score: {predicted_scores[1]:.2f}"
    # )
    # print(
    #     f"Valor real -> blue_score: {true_scores[0]:.2f}, red_score: {true_scores[1]:.2f}"
    # )
    
    # print("Win prediction was", actual_winner == pred_winner)
    return actual_winner == pred_winner

In [155]:
fila_ejemplo = original_df.iloc[301]
inferencia(fila_ejemplo, model, scaler)

Predicción -> blue_score: 46.20, red_score: 66.70
Valor real -> blue_score: 48.00, red_score: 86.00
Win prediction was True


True

In [159]:
train_df, test_df = train_test_split(data_df, test_size=0.15, random_state=42)

correct_wins = 0
for i in range(len(train_df)):
    fila = train_df.iloc[i] 
    win = inferencia(fila, model, scaler, threshold=5.0)
    if win:
        correct_wins += 1
    
print(correct_wins / len(train_df))

0.8568084276238783


In [147]:
torch.save(model.state_dict(), "model.pt")